In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

In [8]:
# load feature matrix
X_min = pd.read_hdf('0_0315_4_electrodes_min.h5')
X_eff = pd.read_hdf('0_0315_4_electrodes_eff.h5')

# load target datafram
y = pd.read_hdf('0_0315_4_electrodes_y.h5')

In [7]:
X_min.head()

variable,27__length,27__maximum,27__mean,27__median,27__minimum,27__standard_deviation,27__sum_values,27__variance,34__length,34__maximum,...,4__sum_values,4__variance,57__length,57__maximum,57__mean,57__median,57__minimum,57__standard_deviation,57__sum_values,57__variance
id,,,,,,,,,,,,,,,,,,,,,
0,6000.0,64.282079,-7.094168,-16.079742,-98.425293,40.795184,-42565.009585,1664.247023,6000.0,53.979924,...,-80267.818929,25313.868698,6000.0,290.661661,-17.225708,-64.398988,-204.070514,153.405547,-103354.247551,23533.261792
1,6000.0,57.034841,-15.617231,-5.975450,-108.570478,56.395064,-93703.384438,3180.403191,6000.0,82.043767,...,-113697.182993,21397.711372,6000.0,274.290223,-17.849974,-57.854440,-181.483743,142.126070,-107099.845392,20199.819801
2,5999.0,84.817419,-11.171965,-0.181217,-116.684697,57.890610,-67020.617478,3351.322706,5999.0,84.238884,...,117103.672237,16018.828457,5999.0,216.386658,23.001735,20.278273,-150.945904,117.418201,137987.408109,13787.033912
3,6000.0,68.549734,18.164751,31.838341,-62.821892,40.613261,108988.504633,1649.436948,6000.0,75.386610,...,78736.316409,5668.518185,6000.0,136.237402,16.418633,25.737367,-93.334961,65.184365,98511.797509,4249.001457
4,6000.0,111.529645,16.008868,-0.210763,-62.825766,46.985019,96053.206487,2207.592015,6000.0,66.437765,...,-9176.557947,3514.979177,6000.0,92.255836,7.437976,13.568825,-98.530345,53.756255,44627.855964,2889.734970


In [ ]:
impute(X_min)
X_filtered = select_features()